#### Import libraries for DiffDock

In [1]:
import torch
import numpy as np

from fairseq import checkpoint_utils, utils, options, tasks
# from fairseq.logging import progress_bar
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
# import ogb
from pathlib import Path
# from sklearn.metrics import roc_auc_score

import sys
from os import path
import importlib

sys.path.append(path.dirname(path.dirname(path.abspath('evaluate.py'))))

# import logging
import argparse
import os

2023-06-28 12:18:11 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX



import fairseq.tasks:  /Users/anabatsh/Desktop/Industrial_Immersion/Transformer-M/fairseq/tasks
import import_tasks with:  fairseq.tasks


In [2]:
from fairseq.tasks import TASK_DATACLASS_REGISTRY, TASK_REGISTRY

TASK_DATACLASS_REGISTRY

{'hubert_pretraining': fairseq.tasks.hubert_pretraining.HubertPretrainingConfig,
 'translation': fairseq.tasks.translation.TranslationConfig,
 'translation_lev': fairseq.tasks.translation_lev.TranslationLevenshteinConfig,
 'language_modeling': fairseq.tasks.language_modeling.LanguageModelingConfig,
 'simul_text_to_text': fairseq.tasks.translation.TranslationConfig,
 'audio_pretraining': fairseq.tasks.audio_pretraining.AudioPretrainingConfig,
 'sentence_prediction': fairseq.tasks.sentence_prediction.SentencePredictionConfig,
 'translation_from_pretrained_xlm': fairseq.tasks.translation_from_pretrained_xlm.TranslationFromPretrainedXLMConfig,
 'audio_finetuning': fairseq.tasks.audio_finetuning.AudioFinetuningConfig,
 'masked_lm': fairseq.tasks.masked_lm.MaskedLMConfig,
 'dummy_lm': fairseq.benchmark.dummy_lm.DummyLMConfig,
 'dummy_masked_lm': fairseq.benchmark.dummy_masked_lm.DummyMaskedLMConfig}

In [3]:
%load_ext autoreload
%autoreload 2

#### Set arguments

In [4]:
# # path to data
# data_path = './datasets/pcq-pos'

# # path to checkpoints, e.g., ./logs/L12
# save_path = './logs/{folder_to_checkpoints}'    

# # set layers=18 for 18-layer model
# layers = 12

# # dimension of hidden layers                                    
# hidden_size = 768

# # dimension of feed-forward layers
# ffn_size = 768

# # number of attention heads
# num_head = 32

# # number of Gaussian Basis kernels
# num_3d_bias_kernel = 128

# # batch size for a single gpu
# batch_size = 256

# dataset_name = "PCQM4M-LSC-V2-3D"				   
# add_3d = "true"

# # ------------------------------
# batch_size = 256
# update_freq = 1
# seed = 1
# clip_norm = 5
# # data_path = './datasets/'
# # save_path = './logs/path_to_ckpts/'
# dropout = 0.0
# act_dropout = 0.1
# attn_dropout = 0.1
# weight_decay = 0.0
# sandwich_ln = "false"
# droppath_prob = 0.1
# noise_scale = 0.2
# mode_prob = "0.2,0.2,0.6"

# dataset_name = "PCQM4M-LSC-V2-3D"
# add_3d = "true"
# no_2d = "false"

# # ------------------------------
# # python evaluate.py 
# # 	--user-dir $(realpath ./Transformer-M)
# # 	--data-path $data_path 
# # 	--num-workers 16
# # 	--ddp-backend=legacy_ddp 
# # 	--dataset-name $dataset_name 
# # 	--batch-size $batch_size 
# # 	--data-buffer-size 20
# # 	--task graph_prediction 
# # 	--criterion graph_prediction 
# # 	--arch transformer_m_base --num-classes 1
# # 	--encoder-layers $layers 
# # 	--encoder-attention-heads $num_head 
# # 	--add-3d 
# # 	--num-3d-bias-kernel $num_3d_bias_kernel
# # 	--encoder-embed-dim $hidden_size 
# # 	--encoder-ffn-embed-dim $ffn_size 
# # 	--droppath-prob $droppath_prob
# # 	--attention-dropout $attn_dropout 
# # 	--act-dropout $act_dropout 
# # 	--dropout $dropout
# # 	--save-dir $save_path 
# # 	--noise-scale $noise_scale 
# # 	--mode-prob $mode_prob 
# # 	--split valid 
# # 	--metric mae

In [5]:
args = argparse.Namespace(
    act_dropout=0.1, activation_fn='gelu', add_3d=True, add_prev_output_tokens=False, 
    all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, 
    amp_scale_window=None, apply_init=True, arch='transformer_m_base', attention_dropout=0.1, 
    azureml_logging=False, batch_size=256, batch_size_valid=256, best_checkpoint_metric='loss', 
    bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, 
    checkpoint_suffix='', clip_norm=0.0, combine_valid_subsets=None, cpu=False, cpu_offload=False, 
    criterion='graph_prediction', curriculum=0, data_buffer_size=20, data_path='./datasets/pcq-pos', 
    dataset_impl=None, dataset_name='PCQM4M-LSC-V2-3D', ddp_backend='legacy_ddp', ddp_comm_hook='none', 
    device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, 
    distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-1, distributed_rank=0, 
    distributed_world_size=1, dropout=0.0, droppath_prob=0.1, edge_type='multi_hop', ema_decay=0.9999, 
    ema_fp32=False, ema_seed_model=None, ema_start_update=0, ema_update_freq=1, empty_cache_freq=0, 
    encoder_attention_heads=32, encoder_embed_dim=768, encoder_ffn_embed_dim=768, encoder_layers=12, 
    encoder_learned_pos=True, encoder_normalize_before=True, eos=2, fast_stat_sync=False, 
    find_unused_parameters=False, finetune_from_model=None, fix_batches_to_gpus=False, 
    fixed_validation_seed=None, force_anneal=None, fp16=False, fp16_init_scale=128, 
    fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, 
    fp32_reduce_scatter=False, gen_subset='test', gradient_as_bucket_view=False, 
    grouped_shuffling=False, heartbeat_timeout=-1, ignore_unused_valid_subsets=False, 
    init_token=None, keep_best_checkpoints=-1, keep_interval_updates=-1, keep_interval_updates_pattern=-1, 
    keep_last_epochs=-1, load_checkpoint_on_all_dp_ranks=False, localsgd_frequency=3, log_file=None, 
    log_format=None, log_interval=100, lr=[0.25], lr_scheduler='fixed', lr_shrink=0.1, max_epoch=0, 
    max_tokens=None, max_tokens_valid=None, max_update=0, max_valid_steps=None, 
    maximize_best_checkpoint_metric=False, memory_efficient_bf16=False, memory_efficient_fp16=False, 
    metric='mae', min_loss_scale=0.0001, mode_prob='0.2,0.2,0.6', model_parallel_size=1, multi_hop_max_dist=5, 
    no_2d=False, no_epoch_checkpoints=False, no_last_checkpoints=False, no_progress_bar=False, 
    no_reshard_after_forward=False, no_save=False, no_save_optimizer_state=False, no_seed_provided=False, 
    no_shuffle=False, no_token_positional_embeddings=False, noise_scale=0.2, not_fsdp_flatten_parameters=False, 
    nprocs_per_node=1, num_3d_bias_kernel=128, num_atoms=4608, num_classes=1, num_edge_dis=128, num_edges=1536, 
    num_in_degree=512, num_out_degree=512, num_segment=2, num_shards=1, num_spatial=512, num_workers=16, 
    on_cpu_convert_precision=False, optimizer=None, optimizer_overrides='{}', pad=1, patience=-1, 
    pipeline_balance=None, pipeline_checkpoint='never', pipeline_chunks=0, pipeline_decoder_balance=None, 
    pipeline_decoder_devices=None, pipeline_devices=None, pipeline_encoder_balance=None, 
    pipeline_encoder_devices=None, pipeline_model_parallel=False, plasma_path='/tmp/plasma', 
    pooler_activation_fn='tanh', profile=False, quantization_config_path=None, required_batch_size_multiple=8, 
    required_seq_len_multiple=1, reset_dataloader=False, reset_logging=False, reset_lr_scheduler=False, 
    reset_meters=False, reset_optimizer=False, restore_file='checkpoint_last.pt', sandwich_ln=False, 
    save_dir='./logs/L12', save_interval=1, save_interval_updates=0, scoring='bleu', seed=1, sent_loss=False, 
    sentence_avg=False, sentence_class_num=2, separator_token=None, shard_id=0, 
    share_encoder_input_output_embed=False, shorten_data_split_list='', shorten_method='none', 
    simul_type=None, skip_invalid_size_inputs_valid_test=False, slowmo_base_algorithm='localsgd', 
    slowmo_momentum=None, split='valid', stop_min_lr=-1.0, stop_time_hours=0, store_ema=False, 
    suppress_crashes=False, task='graph_prediction', tensorboard_logdir=None, threshold_loss_scale=None, 
    tokenizer=None, tpu=False, train_subset='train', unk=3, update_epoch_batch_itr=False, update_freq=[1], 
    update_ordered_indices_seed=False, use_bmuf=False, use_plasma_view=False, use_sharded_state=False, 
    user_dir='/Users/anabatsh/Desktop/Industrial_Immersion/Transformer-M/Transformer-M', valid_subset='valid', 
    validate_after_updates=0, validate_interval=1, validate_interval_updates=0, wandb_project=None, 
    warmup_updates=0, write_checkpoints_asynchronously=False, zero_sharding='none'
)


In [6]:
def eval_(args, use_pretrained, checkpoint_path=None, logger=None):
    
    module_path = getattr(args, "user_dir", None)
    module_parent, module_name = os.path.split(module_path)
    importlib.import_module(module_name)

    tasks_path = os.path.join(module_path, "tasks")
    if os.path.exists(tasks_path):
        from fairseq.tasks import import_tasks
        import_tasks(tasks_path, f"{module_name}.tasks")
        models_path = os.path.join(module_path, "models")
        print('models_path', models_path, os.path.exists(models_path))
        if os.path.exists(models_path):
            from fairseq.models import import_models
            import_models(models_path, f"{module_name}.models")

    cfg = convert_namespace_to_omegaconf(args)
    np.random.seed(cfg.common.seed)
    utils.set_torch_seed(cfg.common.seed)

    # initialize task
    task = tasks.setup_task(cfg.task)
    return
    # model = task.build_model(cfg.model)

    # # load checkpoint

    # model_state = torch.load(checkpoint_path)["model"]
    # model.load_state_dict(
    #     model_state, strict=True, model_cfg=cfg.model
    # )
    # del model_state

    # model.to(torch.cuda.current_device())
    # # load dataset
    # split = args.split
    # task.load_dataset(split)
    # batch_iterator = task.get_batch_iterator(
    #     dataset=task.dataset(split),
    #     max_tokens=cfg.dataset.max_tokens_valid,
    #     max_sentences=cfg.dataset.batch_size_valid,
    #     max_positions=utils.resolve_max_positions(
    #         task.max_positions(),
    #         model.max_positions(),
    #     ),
    #     ignore_invalid_inputs=cfg.dataset.skip_invalid_size_inputs_valid_test,
    #     required_batch_size_multiple=cfg.dataset.required_batch_size_multiple,
    #     seed=cfg.common.seed,
    #     num_workers=cfg.dataset.num_workers,
    #     epoch=0,
    #     data_buffer_size=cfg.dataset.data_buffer_size,
    #     disable_iterator_cache=False,
    # )
    # itr = batch_iterator.next_epoch_itr(
    #     shuffle=False, set_dataset_epoch=False
    # )
    # progress = progress_bar.progress_bar(
    #     itr,
    #     log_format=cfg.common.log_format,
    #     log_interval=cfg.common.log_interval,
    #     default_log_format=("tqdm" if not cfg.common.no_progress_bar else "simple")
    # )

    # # infer
    # y_pred = []
    # y_true = []
    # with torch.no_grad():
    #     model.eval()
    #     for i, sample in enumerate(progress):
    #         sample = utils.move_to_cuda(sample)
    #         y = model(**sample["net_input"])[0][:, 0, :].reshape(-1)
    #         y_pred.extend(y.detach().cpu())
    #         y_true.extend(sample["target"].detach().cpu().reshape(-1)[:y.shape[0]])
    #         torch.cuda.empty_cache()

    # # save predictions
    # y_pred = torch.Tensor(y_pred)
    # y_true = torch.Tensor(y_true)

    # # evaluate pretrained models

    # if args.metric == "auc":
    #     auc = roc_auc_score(y_true, y_pred)
    #     logger.info(f"auc: {auc}")
    #     return auc
    # elif args.metric == "mae":
    #     mae = np.mean(np.abs(y_true.numpy() - y_pred.numpy()))
    #     logger.info(f"mae: {mae}")
    #     return mae
    # else:
    #     raise ValueError(f"Unsupported metric {args.metric}")

In [7]:
# def main():
# parser = options.get_training_parser()

# parser.add_argument(
#     "--split",
#     type=str,
# )
# parser.add_argument(
#     "--metric",
#     type=str,
# )
# args = options.parse_args_and_arch(parser, modify_parser=None)

# logger = logging.getLogger(__name__)
for checkpoint_fname in os.listdir(args.save_dir):
    checkpoint_path = Path(args.save_dir) / checkpoint_fname
    if str(checkpoint_path)[-3:] == '.pt':
        # logger.info(f"evaluating checkpoint file {checkpoint_path}")
        result = eval_(args, False, checkpoint_path, None) #logger)
        open(str(checkpoint_path)[:-3] + f'_{args.split}_{result:.5f}.txt', 'w')

2023-06-28 12:18:12 | INFO | rdkit | Enabling RDKit 2023.03.1 jupyter extensions


import import_tasks with:  Transformer-M.tasks
models_path /Users/anabatsh/Desktop/Industrial_Immersion/Transformer-M/Transformer-M/models True
{'hubert_pretraining': <class 'fairseq.tasks.hubert_pretraining.HubertPretrainingConfig'>, 'translation': <class 'fairseq.tasks.translation.TranslationConfig'>, 'translation_lev': <class 'fairseq.tasks.translation_lev.TranslationLevenshteinConfig'>, 'language_modeling': <class 'fairseq.tasks.language_modeling.LanguageModelingConfig'>, 'simul_text_to_text': <class 'fairseq.tasks.translation.TranslationConfig'>, 'audio_pretraining': <class 'fairseq.tasks.audio_pretraining.AudioPretrainingConfig'>, 'sentence_prediction': <class 'fairseq.tasks.sentence_prediction.SentencePredictionConfig'>, 'translation_from_pretrained_xlm': <class 'fairseq.tasks.translation_from_pretrained_xlm.TranslationFromPretrainedXLMConfig'>, 'audio_finetuning': <class 'fairseq.tasks.audio_finetuning.AudioFinetuningConfig'>, 'masked_lm': <class 'fairseq.tasks.masked_lm.Masked

/Users/anabatsh/miniforge3/envs/indimm/lib/python3.8/site-packages/hydra/experimental/initialize.py:35: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(
/Users/anabatsh/miniforge3/envs/indimm/lib/python3.8/site-packages/hydra/experimental/compose.py:18: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(
/Users/anabatsh/miniforge3/envs/indimm/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'config': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/Users/anabatsh/miniforge3/envs/indimm/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: UserWarning: 
'config' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be remove

TASK <class 'Transformer-M.tasks.graph_prediction.GraphPredictionTask'>
Converting SMILES strings into graphs...


In [ ]:
# cfg = {
#     '_name': None, 
#     'common': {
#         '_name': None, 'no_progress_bar': False, 'log_interval': 100, 
#         'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 
#         'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 
#         'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 
#         'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 
#         'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 
#         'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 
#         'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 
#         'user_dir': '/Users/anabatsh/Desktop/Industrial_Immersion/Transformer-M/Transformer-M', 'empty_cache_freq': 0, 
#         'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 
#         'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 
#         'plasma_path': '/tmp/plasma'
#     }, 
#     'common_eval': {
#         '_name': None, 'path': None, 'post_process': None, 
#         'quiet': False, 'model_overrides': '{}', 'results_path': None
#     }, 
#     'distributed_training': {
#         '_name': None, 'distributed_world_size': 1, 'distributed_num_procs': 1, 
#         'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 
#         'distributed_port': -1, 'device_id': 0, 'distributed_no_spawn': False, 
#         'ddp_backend': 'legacy_ddp', 'ddp_comm_hook': 'none', 'bucket_cap_mb': 25, 
#         'fix_batches_to_gpus': False, 'find_unused_parameters': False, 'gradient_as_bucket_view': False, 
#         'fast_stat_sync': False, 'heartbeat_timeout': -1, 'broadcast_buffers': False, 'slowmo_momentum': None, 
#         'slowmo_base_algorithm': 'localsgd', 'localsgd_frequency': 3, 'nprocs_per_node': 1, 
#         'pipeline_model_parallel': False, 'pipeline_balance': None, 'pipeline_devices': None, 
#         'pipeline_chunks': 0, 'pipeline_encoder_balance': None, 'pipeline_encoder_devices': None, 
#         'pipeline_decoder_balance': None, 'pipeline_decoder_devices': None, 'pipeline_checkpoint': 'never', 
#         'zero_sharding': 'none', 'fp16': False, 'memory_efficient_fp16': False, 'tpu': False, 
#         'no_reshard_after_forward': False, 'fp32_reduce_scatter': False, 'cpu_offload': False, 
#         'use_sharded_state': False, 'not_fsdp_flatten_parameters': False
#     }, 
#     'dataset': {
#         '_name': None, 'num_workers': 16, 'skip_invalid_size_inputs_valid_test': False,
#           'max_tokens': None, 'batch_size': 256, 'required_batch_size_multiple': 8, 
#           'required_seq_len_multiple': 1, 'dataset_impl': None, 'data_buffer_size': 20,
#             'train_subset': 'train', 'valid_subset': 'valid', 'combine_valid_subsets': None, 
#             'ignore_unused_valid_subsets': False, 'validate_interval': 1, 'validate_interval_updates': 0, 
#             'validate_after_updates': 0, 'fixed_validation_seed': None, 'disable_validation': False, 
#             'max_tokens_valid': None, 'batch_size_valid': 256, 'max_valid_steps': None, 'curriculum': 0, 
#             'gen_subset': 'test', 'num_shards': 1, 'shard_id': 0, 'grouped_shuffling': False, 
#             'update_epoch_batch_itr': False, 'update_ordered_indices_seed': False
#     }, 
#     'optimization': {
#         '_name': None, 'max_epoch': 0, 'max_update': 0, 'stop_time_hours': 0.0, 
#         'clip_norm': 0.0, 'sentence_avg': False, 'update_freq': [1], 'lr': [0.25], 
#         'stop_min_lr': -1.0, 'use_bmuf': False
#     }, 
#     'checkpoint': {
#         '_name': None, 'save_dir': './logs/L12', 'restore_file': 'checkpoint_last.pt', 
#         'finetune_from_model': None, 'reset_dataloader': False, 'reset_lr_scheduler': False, 
#         'reset_meters': False, 'reset_optimizer': False, 'optimizer_overrides': '{}', 
#         'save_interval': 1, 'save_interval_updates': 0, 'keep_interval_updates': -1, 
#         'keep_interval_updates_pattern': -1, 'keep_last_epochs': -1, 'keep_best_checkpoints': -1, 
#         'no_save': False, 'no_epoch_checkpoints': False, 'no_last_checkpoints': False, 
#         'no_save_optimizer_state': False, 'best_checkpoint_metric': 'loss', 
#         'maximize_best_checkpoint_metric': False, 'patience': -1, 'checkpoint_suffix': '', 
#         'checkpoint_shard_count': 1, 'load_checkpoint_on_all_dp_ranks': False, 
#         'write_checkpoints_asynchronously': False, 'model_parallel_size': 1
#     }, 
#     'bmuf': {
#         '_name': None, 'block_lr': 1.0, 'block_momentum': 0.875, 'global_sync_iter': 50, 
#         'warmup_iterations': 500, 'use_nbm': False, 'average_sync': False, 'distributed_world_size': 1
#     }, 
#     'generation': {
#         '_name': None, 'beam': 5, 'nbest': 1, 'max_len_a': 0.0, 'max_len_b': 200, 
#         'min_len': 1, 'match_source_len': False, 'unnormalized': False, 'no_early_stop': False, 
#         'no_beamable_mm': False, 'lenpen': 1.0, 'unkpen': 0.0, 'replace_unk': None, 
#         'sacrebleu': False, 'score_reference': False, 'prefix_size': 0, 'no_repeat_ngram_size': 0, 
#         'sampling': False, 'sampling_topk': -1, 'sampling_topp': -1.0, 'constraints': None, 
#         'temperature': 1.0, 'diverse_beam_groups': -1, 'diverse_beam_strength': 0.5, 'diversity_rate': -1.0, 
#         'print_alignment': None, 'print_step': False, 'lm_path': None, 'lm_weight': 0.0, 
#         'iter_decode_eos_penalty': 0.0, 'iter_decode_max_iter': 10, 'iter_decode_force_max_iter': False, 
#         'iter_decode_with_beam': 1, 'iter_decode_with_external_reranker': False, 'retain_iter_history': False, 
#         'retain_dropout': False, 'retain_dropout_modules': None, 'decoding_format': None, 'no_seed_provided': False
#     }, 
#     'eval_lm': {
#         '_name': None, 'output_word_probs': False, 'output_word_stats': False, 'context_window': 0, 
#         'softmax_batch': 9223372036854775807
#     }, 
#     'interactive': {
#         '_name': None, 'buffer_size': 0, 'input': '-'
#     }, 
#     'model': Namespace(
#         _name='transformer_m_base', act_dropout=0.1, activation_fn='gelu', 
#         add_3d=True, add_prev_output_tokens=False, all_gather_list_size=16384, 
#         amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, 
#         apply_init=True, arch='transformer_m_base', attention_dropout=0.1, azureml_logging=False, 
#         batch_size=256, batch_size_valid=256, best_checkpoint_metric='loss', bf16=False, bpe=None, 
#         broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', 
#         clip_norm=0.0, combine_valid_subsets=None, cpu=False, cpu_offload=False, 
#         criterion='graph_prediction', curriculum=0, data_buffer_size=20, data_path='./datasets/pcq-pos', 
#         dataset_impl=None, dataset_name='PCQM4M-LSC-V2-3D', ddp_backend='legacy_ddp', 
#         ddp_comm_hook='none', device_id=0, disable_validation=False, distributed_backend='nccl', 
#         distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-1,
#         distributed_rank=0, distributed_world_size=1, dropout=0.0, droppath_prob=0.1, edge_type='multi_hop', 
#         ema_decay=0.9999, ema_fp32=False, ema_seed_model=None, ema_start_update=0, ema_update_freq=1, 
#         empty_cache_freq=0, encoder_attention_heads=32, encoder_embed_dim=768, encoder_ffn_embed_dim=768, 
#         encoder_layers=12, encoder_learned_pos=True, encoder_normalize_before=True, eos=2, fast_stat_sync=False, 
#         find_unused_parameters=False, finetune_from_model=None, fix_batches_to_gpus=False, 
#         fixed_validation_seed=None, force_anneal=None, fp16=False, fp16_init_scale=128, 
#         fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, fp32_reduce_scatter=False, 
#         gen_subset='test', gradient_as_bucket_view=False, grouped_shuffling=False, heartbeat_timeout=-1,
#         ignore_unused_valid_subsets=False, init_token=None, keep_best_checkpoints=-1, keep_interval_updates=-1, 
#         keep_interval_updates_pattern=-1, keep_last_epochs=-1, load_checkpoint_on_all_dp_ranks=False, 
#         localsgd_frequency=3, log_file=None, log_format=None, log_interval=100, lr=[0.25], 
#         lr_scheduler='fixed', lr_shrink=0.1, max_epoch=0, max_tokens=None, max_tokens_valid=None, 
#         max_update=0, max_valid_steps=None, maximize_best_checkpoint_metric=False, 
#         memory_efficient_bf16=False, memory_efficient_fp16=False, metric='mae', min_loss_scale=0.0001, 
#         mode_prob='0.2,0.2,0.6', model_parallel_size=1, multi_hop_max_dist=5, no_2d=False, 
#         no_epoch_checkpoints=False, no_last_checkpoints=False, no_progress_bar=False, 
#         no_reshard_after_forward=False, no_save=False, no_save_optimizer_state=False, 
#         no_seed_provided=False, no_shuffle=False, no_token_positional_embeddings=False, 
#         noise_scale=0.2, not_fsdp_flatten_parameters=False, nprocs_per_node=1, 
#         num_3d_bias_kernel=128, num_atoms=4608, num_classes=1, num_edge_dis=128, 
#         num_edges=1536, num_in_degree=512, num_out_degree=512, num_segment=2, num_shards=1, 
#         num_spatial=512, num_workers=16, on_cpu_convert_precision=False, optimizer=None, 
#         optimizer_overrides='{}', pad=1, patience=-1, pipeline_balance=None, pipeline_checkpoint='never', 
#         pipeline_chunks=0, pipeline_decoder_balance=None, pipeline_decoder_devices=None, 
#         pipeline_devices=None, pipeline_encoder_balance=None, pipeline_encoder_devices=None, 
#         pipeline_model_parallel=False, plasma_path='/tmp/plasma', pooler_activation_fn='tanh', 
#         profile=False, quantization_config_path=None, required_batch_size_multiple=8, 
#         required_seq_len_multiple=1, reset_dataloader=False, reset_logging=False, 
#         reset_lr_scheduler=False, reset_meters=False, reset_optimizer=False, 
#         restore_file='checkpoint_last.pt', sandwich_ln=False, save_dir='./logs/L12', 
#         save_interval=1, save_interval_updates=0, scoring='bleu', seed=1, sent_loss=False, 
#         sentence_avg=False, sentence_class_num=2, separator_token=None, shard_id=0, 
#         share_encoder_input_output_embed=False, shorten_data_split_list='', shorten_method='none', 
#         simul_type=None, skip_invalid_size_inputs_valid_test=False, slowmo_base_algorithm='localsgd', 
#         slowmo_momentum=None, split='valid', stop_min_lr=-1.0, stop_time_hours=0, store_ema=False, 
#         suppress_crashes=False, task='graph_prediction', tensorboard_logdir=None, threshold_loss_scale=None, 
#         tokenizer=None, tpu=False, train_subset='train', unk=3, update_epoch_batch_itr=False, update_freq=[1], 
#         update_ordered_indices_seed=False, use_bmuf=False, use_plasma_view=False, use_sharded_state=False, 
#         user_dir='/Users/anabatsh/Desktop/Industrial_Immersion/Transformer-M/Transformer-M', 
#         valid_subset='valid', validate_after_updates=0, validate_interval=1, validate_interval_updates=0, 
#         wandb_project=None, warmup_updates=0, write_checkpoints_asynchronously=False, zero_sharding='none'
#     ), 
#     'task': {
#         '_name': 'graph_prediction', 'data_path': './datasets/pcq-pos', 
#         'noise_scale': 0.2, 'sandwich_ln': False, 'num_classes': 1, 'init_token': None, 
#         'separator_token': None, 'no_shuffle': False, 'shorten_method': 'none', 
#         'shorten_data_split_list': '', 'add_prev_output_tokens': False, 'max_positions': 512, 
#         'dataset_name': 'PCQM4M-LSC-V2-3D', 'num_atoms': 4608, 'num_edges': 1536, 
#         'num_in_degree': 512, 'num_out_degree': 512, 'num_spatial': 512, 'num_edge_dis': 128, 
#         'multi_hop_max_dist': 5, 'edge_type': 'multi_hop', 'regression_target': True, 'seed': 1
#     }, 
#     'criterion': {
#         '_name': 'graph_prediction', 'tpu': False
#     }, 
#     'optimizer': None, 
#     'lr_scheduler': {
#         '_name': 'fixed', 'force_anneal': None, 'lr_shrink': 0.1, 
#         'warmup_updates': 0, 'lr': [0.25]
#     }, 
#     'scoring': {
#         '_name': 'bleu', 'pad': 1, 'eos': 2, 'unk': 3
#     }, 
#     'bpe': None, 
#     'tokenizer': None, 
#     'ema': {
#         '_name': None, 'store_ema': False, 'ema_decay': 0.9999, 'ema_start_update': 0, 
#         'ema_seed_model': None, 'ema_update_freq': 1, 'ema_fp32': False
#     }, 
#     'simul_type': None
# }
# TASK <class 'Transformer-M.tasks.graph_prediction.GraphPredictionTask'>